# Notebook for training and generating rules

In [ ]:
import os
import random
import sys

import numpy as np
import torch
from torch import nn
from torch import optim

os.environ['CUBLAS_WORKSPACE_CONFIG'] = ":16:8"

from dataloader import RTDataLoader
from framework import RTFramework
from rule_miner import RuleMiner

# Paths
here = "."
data_dir = os.path.join(here, "../datasets/")

dataset = "family"

dataset_dir = os.path.join(data_dir, dataset)
train_file = os.path.join(dataset_dir, "train.json")
valid_file = os.path.join(dataset_dir, "valid.txt")
test_file = os.path.join(dataset_dir, "test.txt")
entities_file = os.path.join(dataset_dir, "entities.txt")
relations_file = os.path.join(dataset_dir, "relations.txt")
all_file = os.path.join(dataset_dir, "all.txt")
"""Saved paths"""
experiment_dir = os.path.join(here, "../saved", dataset)
# Model checkpoint for continuing training.
checkpoint_dir = os.path.join(experiment_dir, "checkpoint/")
# Directory to save trained model.
model_save_dir = os.path.join(experiment_dir, "model/")
# Options file.
option_file = os.path.join(experiment_dir, "option.txt")
# Model prediction file.
prediction_file = os.path.join(experiment_dir, "prediction.txt")
if not os.path.exists(experiment_dir):
    os.mkdir(experiment_dir)
    os.mkdir(checkpoint_dir)
    os.mkdir(model_save_dir)
"""Other configurations"""
device = "cuda"

In [ ]:
"""Hypterparameters"""
top_k = 10
rank = 3
num_steps = 2
query_embed_dim = 128
num_rnn_layers = 1
rnn_hidden_size = 128
seed = 210224
batch_size = 128
train_epochs = 20
beta_l = 1.
beta_s = 0.8
lr = 0.001
num_sample_batches = 0

# Specify random seed.
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [ ]:
dataloader = RTDataLoader(
    relations_file, entities_file,
    all_file, train_file,
    valid_file, test_file
)

num_relations = dataloader.num_relations
num_operators = dataloader.num_operators
num_entities = dataloader.num_entities

dataloader.id2rel[num_operators] = "self"
dataloader.rel2id["self"] = num_operators

## Train

In [ ]:
miner = RuleMiner(
    rank, num_steps, num_entities,
    num_operators, num_operators,
    query_embed_dim, num_rnn_layers,
    rnn_hidden_size
).to(device)
optimizer = optim.Adam(miner.parameters(), lr=lr)
loss_fn = nn.BCEWithLogitsLoss().to(device)
framework = RTFramework(
    miner, optimizer, dataloader,
    loss_fn, device, ckpt_save_dir=checkpoint_dir
)

In [ ]:
framework.train(top_k, batch_size, num_sample_batches, train_epochs)

## Evaluation

In [ ]:
ckpt_file = os.path.join(checkpoint_dir, "checkpoint.pth.tar")
checkpoint = torch.load(ckpt_file)
miner.load_state_dict(checkpoint['model'])
framework.eval("test", batch_size, top_k)

## Generate rules

In [ ]:
for qq, hh, TT, mat in dataloader.one_epoch("test", 10, shuffle=True):
    break
qq = torch.from_numpy(qq).to(device)
hh = torch.from_numpy(hh).to(device)
TT = torch.from_numpy(TT).to(device)
logits = miner(qq, hh, TT, mat)
print([dataloader.id2rel[rel.item()] for rel in qq])

In [ ]:
relation = 4
print(miner.attention_operator_list[0][:, :, relation, -1].size())
print(miner.attention_operator_list[0][:, :, relation, -1])
print(dataloader.id2rel)

In [ ]:
import itertools

attn_combs = [list(range(num_operators+1)) for _ in range(num_steps)]
attn_combs = itertools.product(*attn_combs)
path_rank = []
for comb in attn_combs:
    cur_path = [[], 0.]
    step2rel = list(zip(range(len(comb)), comb))
    for r in range(rank):
        attention_operators = miner.attention_operator_list[r][:, :, relation, -1]
        tmp_score = 1.
        for step, rel in step2rel:
            if r == 0:
                cur_path[0].append(dataloader.id2rel[rel])
            tmp_score *= attention_operators[step, rel].item()
        cur_path[1] += tmp_score
    path_rank.append(cur_path)
path_rank.sort(key=lambda x: x[1], reverse=True)

In [ ]:
path_rank[:10]